## VN30F1m - 5m - Fibonaci - Analytics

In [14]:
import warnings
warnings.filterwarnings('ignore')

import time
from datetime import date, datetime
from datetime import timedelta
from dateutil.relativedelta import relativedelta

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [15]:
import os
from pathlib import Path
notebook_path = os.getcwd()
algo_dir = Path(notebook_path).parent.parent
csv_file = str(algo_dir) + '/vn-stock-data/VN30ps/VN30F1M_5minutes.csv'
is_file = os.path.isfile(csv_file)
if is_file:
    dataset = pd.read_csv(csv_file, index_col='Date', parse_dates=True)
else:
    dataset = pd.read_csv("https://raw.githubusercontent.com/zuongthaotn/vn-stock-data/main/VN30ps/VN30F1M_5minutes.csv", index_col='Date', parse_dates=True)

In [16]:
_1_d_df = dataset.copy()
_1_d_df = _1_d_df.resample("D").agg({
    'Open': 'first',
    'Close': 'last',
    'High': 'max',
    'Low': 'min'
})
_1_d_df.dropna(inplace=True)
_1_d_df['High_s'] = _1_d_df['High'].shift(1)
_1_d_df['Low_s'] = _1_d_df['Low'].shift(1)
_1_d_df['Close_s'] = _1_d_df['Close'].shift(1)
_1_d_df['Pivot'] = _1_d_df.apply(
        lambda row: (row['High_s'] + row['Low_s'] + row['Close_s']) / 3, axis=1)
_1_d_df = _1_d_df[['Pivot']]

In [17]:
data = dataset.copy()
data = data.assign(time_d=pd.PeriodIndex(data.index, freq='1D').to_timestamp())
data = pd.merge(data, _1_d_df, left_on="time_d", right_index=True, how="left")
data.dropna(inplace=True)
data

,Open,High,Low,Close,Volume,time_d,Pivot
Date,,,,,,,
2018-08-14 09:00:00,955.5,955.5,954.7,954.9,1103,2018-08-14,950.366667
2018-08-14 09:05:00,954.8,955.0,954.3,955.0,530,2018-08-14,950.366667
2018-08-14 09:10:00,955.0,955.1,954.7,955.0,509,2018-08-14,950.366667
2018-08-14 09:15:00,955.0,957.0,955.0,956.9,1758,2018-08-14,950.366667
2018-08-14 09:20:00,956.5,956.6,955.6,955.8,1230,2018-08-14,950.366667
...,...,...,...,...,...,...,...
2024-06-19 14:15:00,1320.6,1325.0,1319.0,1322.7,15368,2024-06-19,1313.133333
2024-06-19 14:20:00,1322.6,1323.0,1317.0,1317.0,12067,2024-06-19,1313.133333
2024-06-19 14:25:00,1317.3,1318.0,1314.3,1314.5,11350,2024-06-19,1313.133333


In [18]:
data["R"] = data["Pivot"] + 6.18
data["S"] = data["Pivot"] - 6.18
data["prev_Close"] = data["Close"].shift(1)
data = data[100*data.index.hour + data.index.minute < 1430]

In [35]:
# Calculate return
data['return'] = ''
data['signal'] = ''
data['exit_time'] = ''
for i, row in data.iterrows():
    if row['prev_Close'] < row['R'] < row['Close']:
        # Long signal
        data.at[i, 'signal'] = 'long'
        current_date = row.name.strftime('%Y-%m-%d ').format()
        current_time = row.name
        entry_price = row['Close']
        data_to_end_day = data[(data.index > current_time) & (data.index < current_date+' 14:30:00')]
        max_price = 0
        exit_time = ''
        for k, wrow in data_to_end_day.iterrows():
            if wrow['Close'] < entry_price and wrow['Close'] < entry_price - 1.5:
                # Stop loss
                momentum = wrow['Close'] - entry_price
                exit_time = wrow.name
                break
            else:
                if wrow['Close'] > entry_price + 5:
                    # Take profit
                    momentum = wrow['Close'] - entry_price
                    exit_time = wrow.name
                    break
                else:
                    # Close at 02:25PM
                    momentum = wrow['Close'] - entry_price
                    exit_time = wrow.name
        data.at[i, 'return'] = momentum
        data.at[i, 'exit_time'] = exit_time
    elif row['prev_Close'] > row['S'] > row['Close']:
        # Short signal
        data.at[i, 'signal'] = 'short'
        current_date = row.name.strftime('%Y-%m-%d ').format()
        current_time = row.name
        entry_price = row['Close']
        data_to_end_day = data[(data.index > current_time) & (data.index < current_date+' 14:30:00')]
        min_price = 10000
        exit_time = ''
        for k, wrow in data_to_end_day.iterrows():
            if wrow['Close'] > entry_price and wrow['Close'] > entry_price + 1.5:
                # Stop loss
                momentum = entry_price - wrow['Close']
                exit_time = wrow.name
                break
            else:
                if wrow['Close'] < entry_price - 5:
                    # Take profit
                    momentum = entry_price - wrow['Close']
                    exit_time = wrow.name
                    break
                else:
                    # Close at 02:25PM
                    momentum = entry_price - wrow['Close']
                    exit_time = wrow.name
        data.at[i, 'return'] = momentum
        data.at[i, 'exit_time'] = exit_time

In [36]:
# Today data
selected_date_data = data[(data.index > '2024-06-19 00:00:00') & (data.index < '2024-06-19 15:00:00')]
selected_date_data

,Open,High,Low,Close,Volume,time_d,Pivot,R,S,prev_Close,return,signal,exit_time
Date,,,,,,,,,,,,,
2024-06-19 09:00:00,1309.8,1311.0,1309.8,1310.9,5161,2024-06-19,1313.133333,1319.313333,1306.953333,1308.5,,,
2024-06-19 09:05:00,1311.1,1311.9,1310.9,1311.7,2895,2024-06-19,1313.133333,1319.313333,1306.953333,1310.9,,,
2024-06-19 09:10:00,1311.7,1311.8,1311.2,1311.6,2166,2024-06-19,1313.133333,1319.313333,1306.953333,1311.7,,,
2024-06-19 09:15:00,1311.5,1311.9,1310.0,1311.1,3993,2024-06-19,1313.133333,1319.313333,1306.953333,1311.6,,,
2024-06-19 09:20:00,1311.1,1311.8,1310.7,1311.5,1642,2024-06-19,1313.133333,1319.313333,1306.953333,1311.1,,,
2024-06-19 09:25:00,1311.5,1311.6,1310.3,1310.6,2173,2024-06-19,1313.133333,1319.313333,1306.953333,1311.5,,,
2024-06-19 09:30:00,1310.6,1310.6,1308.5,1309.0,4184,2024-06-19,1313.133333,1319.313333,1306.953333,1310.6,,,
2024-06-19 09:35:00,1308.9,1310.5,1308.9,1310.0,3294,2024-06-19,1313.133333,1319.313333,1306.953333,1309.0,,,
2024-06-19 09:40:00,1310.1,1310.3,1309.2,1309.3,2588,2024-06-19,1313.133333,1319.313333,1306.953333,1310.0,,,


In [37]:
has_return = data[data['return'] != '']

In [39]:
has_return

,Open,High,Low,Close,Volume,time_d,Pivot,R,S,prev_Close,return,signal,exit_time
Date,,,,,,,,,,,,,
2018-08-14 09:15:00,955.0,957.0,955.0,956.9,1758,2018-08-14,950.366667,956.546667,944.186667,955.0,-3.4,long,2018-08-14 10:35:00
2018-08-14 10:15:00,956.6,957.3,956.3,957.1,1335,2018-08-14,950.366667,956.546667,944.186667,956.5,-1.7,long,2018-08-14 10:30:00
2018-08-14 13:25:00,955.8,956.7,955.8,956.7,1576,2018-08-14,950.366667,956.546667,944.186667,955.9,3.4,long,2018-08-14 14:25:00
2018-08-15 14:15:00,954.4,954.4,951.3,951.5,4130,2018-08-15,957.733333,963.913333,951.553333,954.1,0.3,short,2018-08-15 14:25:00
2018-08-16 09:00:00,942.4,942.4,941.0,942.0,1666,2018-08-16,951.833333,958.013333,945.653333,946.6,5.2,short,2018-08-16 10:55:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-06-18 10:50:00,1315.8,1316.1,1315.5,1315.9,1953,2024-06-18,1309.700000,1315.880000,1303.520000,1315.8,5.5,long,2024-06-18 13:55:00
2024-06-18 11:05:00,1315.3,1316.4,1315.1,1316.3,2193,2024-06-18,1309.700000,1315.880000,1303.520000,1315.3,5.1,long,2024-06-18 13:55:00
2024-06-18 11:20:00,1314.9,1316.1,1314.9,1315.9,2283,2024-06-18,1309.700000,1315.880000,1303.520000,1315.0,5.5,long,2024-06-18 13:55:00


In [38]:
has_return['return'].sum()

400.09999999999707